# 1 - Introduction

This notebook train four different neural network models. They are all pretty similar.

## 1.1 Load Packages and Global Variables

In [1]:
%matplotlib inline
import os
import numpy as np
import nltk
from sklearn.externals import joblib
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
PROJECT_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(PROJECT_DIR)

In [3]:
os.getcwd()

'C:\\Users\\wertu\\Documents\\Datascience\\udacity-ml-capstone'

In [4]:
import src.neural_networks as nn
from src.evaluation import roc_plot

In [5]:
from importlib import reload
nn = reload(nn)

## 1.2- Load the Data

Load the train and validation neural network data sets if they are present, otherwise raise an exception.

In [6]:
#Load sequnces
try:
    train = joblib.load('data/processed/train_nn.pkl')
    valid = joblib.load('data/processed/valid_nn.pkl')
    test = joblib.load('data/processed/test_nn.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [7]:
#Load embedding matrix of 50 dimensions
try:
    embedding_matrix50 = joblib.load('data/interim/embeddings50.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [8]:
#Load embedding matrix of 300 dimensions
try:
    embedding_matrix300 = joblib.load('data/interim/embeddings300.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [9]:
#these arguments will be the same when training both models
run_args = {"train":(train["seqs"],train["labels"]),
            "valid":(valid["seqs"],valid["labels"])}

# 4 - Train 50d model with trainable embeds

Will now make embedding layer of first model trainable. Dropout of inputs to recurrent layer is automatically increased when the embedding layer is trainable

In [22]:
model50_trained = nn.create_model(n_hidden=150, embedding_matrix=embedding_matrix50, train_embed=True)

In [23]:
#Load weights
model50_trained.load_weights('models/nn_50d_fixed.hdf5')
model50_trained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 50)           1369800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 50)           0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 50)           0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 50)           0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               120600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [24]:
#now train
#decrease learning rate from default of 0.001
#increase patience to compensae to 50 from 25
#and remove calculation of training auc to speed up
history50_trained = nn.run_model(model=model50_trained,out_path="models/nn_50d_trainedv2.hdf5",
                                 patience=50, train_auc=False, **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6274 - acc: 0.6472 val_auc: 73.1424%
Epoch 00001: val_auc improved from -inf to 0.73142, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 177s 1ms/step - loss: 0.6273 - acc: 0.6474 - val_loss: 0.6097 - val_acc: 0.6604
Epoch 2/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6253 - acc: 0.6479 val_auc: 73.0896%
Epoch 00002: val_auc did not improve
171945/171945 [==============================] - 106s 618us/step - loss: 0.6251 - acc: 0.6481 - val_loss: 0.6088 - val_acc: 0.6616
Epoch 3/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6232 - acc: 0.6520 val_auc: 73.0396%
Epoch 00003: val_auc did not improve
171945/171945 [==============================] - 104s 603us/step - loss: 0.6231 - acc: 0.6520 - val_loss: 0.6091 - val_acc: 0.6604
Epoch 4/1000
170000/171945 [==========

Epoch 54/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6086 - acc: 0.6669 val_auc: 73.4592%
Epoch 00054: val_auc did not improve
171945/171945 [==============================] - 100s 584us/step - loss: 0.6086 - acc: 0.6669 - val_loss: 0.6097 - val_acc: 0.6630
Epoch 55/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6072 - acc: 0.6690 val_auc: 73.4497%
Epoch 00055: val_auc did not improve
171945/171945 [==============================] - 103s 599us/step - loss: 0.6074 - acc: 0.6688 - val_loss: 0.6095 - val_acc: 0.6635
Epoch 56/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6085 - acc: 0.6665 val_auc: 73.4720%
Epoch 00056: val_auc improved from 0.73461 to 0.73472, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 104s 605us/step - loss: 0.6083 - acc: 0.6667 - val_loss: 0.6082 - val_acc: 0.6652
Epoch 57/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.

Epoch 80/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6008 - acc: 0.6754 val_auc: 73.8341%
Epoch 00080: val_auc did not improve
171945/171945 [==============================] - 103s 598us/step - loss: 0.6009 - acc: 0.6754 - val_loss: 0.6078 - val_acc: 0.6670
Epoch 81/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5998 - acc: 0.6772 val_auc: 73.8630%
Epoch 00081: val_auc improved from 0.73838 to 0.73863, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 103s 598us/step - loss: 0.5999 - acc: 0.6771 - val_loss: 0.6065 - val_acc: 0.6680
Epoch 82/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.6001 - acc: 0.6777 val_auc: 73.9052%
Epoch 00082: val_auc improved from 0.73863 to 0.73905, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 103s 601us/step - loss: 0.6002 - acc: 0.6777 - val_loss: 0.6056 - val_acc: 0.6683
Epoch 83/1000
170000

Epoch 106/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5934 - acc: 0.6831 val_auc: 74.2135%
Epoch 00106: val_auc did not improve
171945/171945 [==============================] - 101s 587us/step - loss: 0.5933 - acc: 0.6833 - val_loss: 0.6068 - val_acc: 0.6687
Epoch 107/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5933 - acc: 0.6826 val_auc: 74.2290%
Epoch 00107: val_auc improved from 0.74223 to 0.74229, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 101s 588us/step - loss: 0.5933 - acc: 0.6826 - val_loss: 0.6058 - val_acc: 0.6696
Epoch 108/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5922 - acc: 0.6847 val_auc: 74.2218%
Epoch 00108: val_auc did not improve
171945/171945 [==============================] - 101s 587us/step - loss: 0.5921 - acc: 0.6847 - val_loss: 0.6049 - val_acc: 0.6711
Epoch 109/1000
170000/171945 [============================>.] - ETA: 0s - loss

Epoch 132/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5849 - acc: 0.6909 val_auc: 74.5640%
Epoch 00132: val_auc improved from 0.74563 to 0.74564, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 100s 583us/step - loss: 0.5850 - acc: 0.6908 - val_loss: 0.6036 - val_acc: 0.6746
Epoch 133/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5850 - acc: 0.6919 val_auc: 74.5504%
Epoch 00133: val_auc did not improve
171945/171945 [==============================] - 100s 582us/step - loss: 0.5850 - acc: 0.6920 - val_loss: 0.6057 - val_acc: 0.6729
Epoch 134/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5844 - acc: 0.6918 val_auc: 74.5857%
Epoch 00134: val_auc improved from 0.74564 to 0.74586, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 100s 582us/step - loss: 0.5845 - acc: 0.6918 - val_loss: 0.6042 - val_acc: 0.6747
Epoch 135/1000
17

...

170000/171945 [============================>.] - ETA: 0s - loss: 0.5587 - acc: 0.7141 val_auc: 75.3198%
Epoch 00210: val_auc did not improve
171945/171945 [==============================] - 100s 583us/step - loss: 0.5588 - acc: 0.7141 - val_loss: 0.6056 - val_acc: 0.6840
Epoch 211/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5594 - acc: 0.7129 val_auc: 75.3542%
Epoch 00211: val_auc improved from 0.75340 to 0.75354, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 100s 584us/step - loss: 0.5594 - acc: 0.7130 - val_loss: 0.6052 - val_acc: 0.6850
Epoch 212/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5597 - acc: 0.7146 val_auc: 75.3430%
Epoch 00212: val_auc did not improve
171945/171945 [==============================] - 101s 585us/step - loss: 0.5598 - acc: 0.7145 - val_loss: 0.6046 - val_acc: 0.6855
Epoch 213/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5597 - acc:

Epoch 237/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5509 - acc: 0.7203 val_auc: 75.5245%
Epoch 00237: val_auc did not improve
171945/171945 [==============================] - 103s 601us/step - loss: 0.5508 - acc: 0.7203 - val_loss: 0.6044 - val_acc: 0.6885
Epoch 238/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5522 - acc: 0.7193 val_auc: 75.5341%
Epoch 00238: val_auc did not improve
171945/171945 [==============================] - 103s 600us/step - loss: 0.5520 - acc: 0.7195 - val_loss: 0.6038 - val_acc: 0.6890
Epoch 239/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5512 - acc: 0.7206 val_auc: 75.5472%
Epoch 00239: val_auc did not improve
171945/171945 [==============================] - 103s 599us/step - loss: 0.5512 - acc: 0.7205 - val_loss: 0.6054 - val_acc: 0.6882
Epoch 240/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5505 - acc: 0.7205 val_auc: 75.5370%
Epoch 00240: val_auc

Epoch 264/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5433 - acc: 0.7255 val_auc: 75.6890%
Epoch 00264: val_auc did not improve
171945/171945 [==============================] - 103s 601us/step - loss: 0.5432 - acc: 0.7256 - val_loss: 0.6052 - val_acc: 0.6893
Epoch 265/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5430 - acc: 0.7257 val_auc: 75.7485%
Epoch 00265: val_auc improved from 0.75719 to 0.75748, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 104s 602us/step - loss: 0.5431 - acc: 0.7257 - val_loss: 0.6056 - val_acc: 0.6890
Epoch 266/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5428 - acc: 0.7267 val_auc: 75.7372%
Epoch 00266: val_auc did not improve
171945/171945 [==============================] - 103s 599us/step - loss: 0.5429 - acc: 0.7266 - val_loss: 0.6057 - val_acc: 0.6891
Epoch 267/1000
170000/171945 [============================>.] - ETA: 0s - loss

Epoch 319/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5263 - acc: 0.7392 val_auc: 75.8845%
Epoch 00319: val_auc did not improve
171945/171945 [==============================] - 100s 584us/step - loss: 0.5264 - acc: 0.7391 - val_loss: 0.6105 - val_acc: 0.6902
Epoch 320/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5264 - acc: 0.7383 val_auc: 75.8387%
Epoch 00320: val_auc did not improve
171945/171945 [==============================] - 101s 585us/step - loss: 0.5264 - acc: 0.7383 - val_loss: 0.6103 - val_acc: 0.6900
Epoch 321/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5245 - acc: 0.7394 val_auc: 75.8987%
Epoch 00321: val_auc did not improve
171945/171945 [==============================] - 101s 585us/step - loss: 0.5246 - acc: 0.7393 - val_loss: 0.6107 - val_acc: 0.6899
Epoch 322/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5246 - acc: 0.7395 val_auc: 75.9056%
Epoch 00322: val_auc

Epoch 376/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5080 - acc: 0.7508 val_auc: 76.0356%
Epoch 00376: val_auc did not improve
171945/171945 [==============================] - 101s 585us/step - loss: 0.5080 - acc: 0.7508 - val_loss: 0.6159 - val_acc: 0.6909
Epoch 377/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5093 - acc: 0.7508 val_auc: 76.0851%
Epoch 00377: val_auc improved from 0.76081 to 0.76085, saving model to models/nn_50d_trained.hdf5
171945/171945 [==============================] - 100s 584us/step - loss: 0.5093 - acc: 0.7508 - val_loss: 0.6156 - val_acc: 0.6913
Epoch 378/1000
170000/171945 [============================>.] - ETA: 0s - loss: 0.5069 - acc: 0.7513 val_auc: 75.9743%
Epoch 00378: val_auc did not improve
171945/171945 [==============================] - 101s 585us/step - loss: 0.5069 - acc: 0.7513 - val_loss: 0.6178 - val_acc: 0.6902
Epoch 379/1000
170000/171945 [============================>.] - ETA: 0s - loss

In [25]:
#save history
joblib.dump(history50_trained.history, "data/interim/nn50_trained_historyv2.pkl")

['data/interim/nn50_trained_history.pkl']

# 5 - Train 300d model with trainable embeds

In [26]:
#create model
model300_trained = nn.create_model(embedding_matrix=embedding_matrix300, n_hidden=150, train_embed=True)
model300_trained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 300)          8218800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 300)          0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 300)          0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 300)          0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               270600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [21]:
#load in weights
model300_trained.load_weights('models/nn_300d_fixed.hdf5')

In [27]:
%%time
history300_trained = nn.run_model(model=model300_trained, out_path="models/nn_300d_trainedv2.hdf5",
                                  patience=50, train_auc=False, **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6812 - acc: 0.5655 val_auc: 64.5726%
Epoch 00001: val_auc improved from -inf to 0.64573, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 197s 1ms/step - loss: 0.6811 - acc: 0.5658 - val_loss: 0.6634 - val_acc: 0.5984
Epoch 2/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6629 - acc: 0.5978 val_auc: 65.8839%
Epoch 00002: val_auc improved from 0.64573 to 0.65884, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 666us/step - loss: 0.6627 - acc: 0.5980 - val_loss: 0.6555 - val_acc: 0.6136
Epoch 3/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6575 - acc: 0.6047 val_auc: 66.5378%
Epoch 00003: val_auc improved from 0.65884 to 0.66538, saving model to models/nn_300d_trained.hdf5
171945/171945 [=============================

Epoch 25/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6190 - acc: 0.6602 val_auc: 71.6608%
Epoch 00025: val_auc did not improve
171945/171945 [==============================] - 115s 668us/step - loss: 0.6190 - acc: 0.6603 - val_loss: 0.6229 - val_acc: 0.6600
Epoch 26/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6164 - acc: 0.6633 val_auc: 71.8314%
Epoch 00026: val_auc improved from 0.71788 to 0.71831, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 667us/step - loss: 0.6162 - acc: 0.6636 - val_loss: 0.6213 - val_acc: 0.6622
Epoch 27/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.6145 - acc: 0.6665 val_auc: 71.8461%
Epoch 00027: val_auc improved from 0.71831 to 0.71846, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 667us/step - loss: 0.6146 - acc: 0.6664 - val_loss: 0.6217 - val_acc: 0.6621
Epoch 28/1000
1700

Epoch 74/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5696 - acc: 0.7064 val_auc: 73.9339%
Epoch 00074: val_auc improved from 0.73889 to 0.73934, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 667us/step - loss: 0.5696 - acc: 0.7063 - val_loss: 0.6141 - val_acc: 0.6765
Epoch 75/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5693 - acc: 0.7065 val_auc: 73.9792%
Epoch 00075: val_auc improved from 0.73934 to 0.73979, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 667us/step - loss: 0.5693 - acc: 0.7064 - val_loss: 0.6131 - val_acc: 0.6772
Epoch 76/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5668 - acc: 0.7077 val_auc: 73.9983%
Epoch 00076: val_auc improved from 0.73979 to 0.73998, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 668us/step - loss: 0.5666 - acc: 

Epoch 100/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5488 - acc: 0.7212 val_auc: 74.3045%
Epoch 00100: val_auc did not improve
171945/171945 [==============================] - 115s 667us/step - loss: 0.5487 - acc: 0.7214 - val_loss: 0.6176 - val_acc: 0.6796
Epoch 101/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5485 - acc: 0.7206 val_auc: 74.3060%
Epoch 00101: val_auc did not improve
171945/171945 [==============================] - 115s 666us/step - loss: 0.5485 - acc: 0.7207 - val_loss: 0.6176 - val_acc: 0.6797
Epoch 102/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5468 - acc: 0.7223 val_auc: 74.3434%
Epoch 00102: val_auc improved from 0.74312 to 0.74343, saving model to models/nn_300d_trained.hdf5
171945/171945 [==============================] - 115s 668us/step - loss: 0.5468 - acc: 0.7223 - val_loss: 0.6193 - val_acc: 0.6795
Epoch 103/1000
170000/171945 [============================>.] - ETA: 1s - los

Epoch 156/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5037 - acc: 0.7500 val_auc: 74.2853%
Epoch 00156: val_auc did not improve
171945/171945 [==============================] - 115s 667us/step - loss: 0.5038 - acc: 0.7499 - val_loss: 0.6505 - val_acc: 0.6826
Epoch 157/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5028 - acc: 0.7520 val_auc: 74.2937%
Epoch 00157: val_auc did not improve
171945/171945 [==============================] - 115s 668us/step - loss: 0.5029 - acc: 0.7519 - val_loss: 0.6520 - val_acc: 0.6814
Epoch 158/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5022 - acc: 0.7515 val_auc: 74.3251%
Epoch 00158: val_auc did not improve
171945/171945 [==============================] - 115s 667us/step - loss: 0.5023 - acc: 0.7514 - val_loss: 0.6529 - val_acc: 0.6838
Epoch 159/1000
170000/171945 [============================>.] - ETA: 1s - loss: 0.5017 - acc: 0.7525 val_auc: 74.2922%
Epoch 00159: val_auc

In [28]:
#save history
joblib.dump(history300_trained.history, "data/interim/nn300_trained_historyv2.pkl")

['data/interim/nn300_trained_history.pkl']

# Analyze

In [ ]:
#load histories
model_names = ("nn50_fixed","nn300_fixed","nn50_trainedv2","nn300_trainedv2")
hist_list = [joblib.load("data/interim/"+hist+"_history.pkl") for hist in model_names]